In [ ]:
using LightGraphs, MetaGraphs, GraphUtils
using Parameters
using LinearAlgebra
using Compose
using Colors
using TaskGraphs
using GraphPlottingBFS
using DataStructures
using JuMP, MathOptInterface
using Gurobi
using CRCBS
using WebotsSim

In [1]:
N = 4                  # num robots
M = 6                  # num delivery tasks
# project_spec = construct_random_project_spec(M;max_parents=3,depth_bias=1.0,Δt_min=0,Δt_max=0)
factory_graph = initialize_regular_grid_graph(;n_obstacles_x=1,n_obstacles_y=1)
vtxs = collect(vertices(factory_graph))
dist_matrix = get_dist_matrix(factory_graph)
r₀,s₀,sₜ,pts = initialize_random_2D_task_graph_env(N,M;vtxs=vtxs,d=[10,10])
Drs, Dss = cached_pickup_and_delivery_distances(pts[r₀],pts[s₀],pts[sₜ],(x1,x2)->dist_matrix[x1,x2])
object_ICs = Dict{Int,OBJECT_AT}(o => OBJECT_AT(o,s₀[o]) for o in 1:M)
object_FCs = Dict{Int,OBJECT_AT}(o => OBJECT_AT(o,sₜ[o]) for o in 1:M)
robot_ICs = Dict{Int,ROBOT_AT}(r => ROBOT_AT(r,r₀[r]) for r in 1:N)
project_spec = construct_random_project_spec(M,object_ICs,object_FCs;max_parents=3,depth_bias=1.0,Δt_min=0,Δt_max=0)
plot_graph_bfs(project_spec.graph;mode=nothing,fillcolor="orange")

UndefVarError: UndefVarError: initialize_regular_grid_graph not defined

In [ ]:
delivery_graph = construct_delivery_graph(project_spec,M)
G = delivery_graph.graph
# initialize vector of operation times
Δt = zeros(nv(G)) # Δt[j] is the wait time for the object j to appear once all inputs have been satisfied
for op in project_spec.operations
    for id in get_output_ids(op)
        Δt[id] = duration(op)
    end
end
# set initial conditions
to0_ = Dict{Int,Float64}()
for v in vertices(G)
    if is_leaf_node(G,v)
        to0_[v] = 0.0
    end
end
tr0_ = Dict{Int,Float64}()
for i in 1:N
    tr0_[i] = 0.0
end
plot_graph_bfs(delivery_graph.graph;mode=nothing)

In [ ]:
model = formulate_JuMP_optimization_problem(G,Drs,Dss,Δt,to0_,tr0_,Gurobi.Optimizer;OutputFlag=0);
start_time = time()
optimize!(model)
solve_time = time() - start_time
optimal = (termination_status(model) == MathOptInterface.TerminationStatusCode(1))
@show solve_time
@show optimal;
assignment = Matrix{Int}(value.(model[:x]));

In [ ]:
spec = TaskGraphProblemSpec(N,M,G,Drs,Dss,Δt,tr0_,to0_)
cache = SearchCache(N,M,to0_,tr0_)
for j in 1:M
    i = findfirst(assignment[:,j] .== 1)
    cache.x[i,j] = 1
end
solution_graph = construct_solution_graph(delivery_graph.graph,assignment)
cache = process_solution(model,cache,spec); # process_solution_fast(model,cache,spec)
# assignments[j] == i ==> robot i performs task j
assignments = map(j->findfirst(cache.x[:,j] .== 1),1:M)
@show assignments;
for r in N+1:N+M
    robot_ICs[r] = ROBOT_AT(r,sₜ[r-N])
end
project_schedule = construct_project_schedule(project_spec, object_ICs, object_FCs, robot_ICs, assignments);

In [ ]:
rg = get_display_metagraph(project_schedule;verbose=false)
plot_graph_bfs(rg)

In [ ]:
task_sequences = Dict{Int,Vector{Int}}()
for agent_id in 1:N
    i = agent_id
    seq = Vector{Int}()
    j = 1
    while j <= M
        if cache.x[i,j] == 1
            push!(seq, j)
            i = j + N
            j = 0
        end
        j += 1
    end
    task_sequences[agent_id] = seq
end
task_sequences

Convert `project_schedule` into a format that can be used within a CRCBS path-finding algorithm

In [ ]:
# Add precedence constraint associated with the order in which robots complete tasks
task_precedence_graph = deepcopy(delivery_graph.graph)
task_to_agent = Dict()
for (agent_idx, seq) in task_sequences
    for (i,v) in enumerate(seq)
        task_to_agent[v] = agent_idx
        if i < length(seq)
            add_edge!(task_precedence_graph, v, seq[i+1])
        end
    end
end
task_precedence_graph

In [ ]:
start_vtxs = r₀
goal_vtxs = map(i->map(i->[s₀[i],sₜ[i]],task_sequences[i]),1:N)
multi_stage_heuristic = MultiStagePerfectHeuristic(G, goal_vtxs)
T = 2*Int(maximum(cache.tof)) # time horizon
conflict_table = HardConflictTable(G,T,N)
mapf = PC_MAPF(
    env = CBS.LowLevelEnv(graph=factory_graph),
    task_precedence_graph = task_precedence_graph,
    num_agents = N,
    agent_ids = map(i->task_to_agent[i], 1:M),
    start_times = Int.(cache.to0), # these might be wrong - need start times for each path (including travel between tasks)
    deadlines = Int.(cache.tof),
    durations = Int.((cache.x .* (Drs))'*ones(N+M) .+ map(j->Dss[j,j],1:M)),
    starts = r₀,
    goals = goal_vtxs 
)

In [ ]:
goal_vtxs = Vector{Vector{Int}}()
for (agent_id,seq) in task_sequences
    for (i,v) in enumerate(seq)
        push!(goal_vtxs, [s₀[i],sₜ[i]])
        if i < length(seq)
            push!(goal_vtxs, [s₀[i+1]])
        end
    end
end
goal_vtxs

In [ ]:
adjacency_matrix(task_precedence_graph)

In [ ]:
render_grid_graph(factory_graph,map(i->[mapf.starts[i], vcat(mapf.goals[i]...)...],1:N))